In [1]:
import numpy as np
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11668\2936652086.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


conclusion: 

Feature Selection: CCI, ADX, Chaikin A/D
Model Selection: best using SVC with poly kernel
Hyperparameter Tuning: d = 2 c = 0.3

Report

1. SVM -> stock price -> goblok tebak all up or down
2. Data collection -> jelasin features dari alphavantage + feature buatan sendiri (short SMA / long SMA ratio)
3. Model Selection -> analyze correlation  bilkang not linearly separable ...
4. Feature Selection
5. Hyperparameter tuning

stevan: 
- feature buatan sendiri (short SMA / long SMA ratio)
- training data selection
- improve acc to >70%

In [2]:
'''
CONST
'''
FILEPATH_STOCK_PRICE = "./tr_eikon_eod_data.csv"
STOCK_NAMES = ['AAPL.O', 'MSFT.O', 'GS.N', 'AMZN.O', 'GS.N']

In [3]:
''' 
UTILS
'''
def sequential_train_test_split(data, ratio = 0.8):
    df = pd.DataFrame(data.dropna()).to_numpy()
    split_index = int(len(df) * ratio)
    return df[:split_index], df[split_index:]

def calculate_return(prediction, data) :
    data = data
    prediction = pd.DataFrame(prediction)
    temp = pd.DataFrame(data)
    temp['log_return'] = np.log(temp / temp.shift(1))
    temp['returne'] = prediction.shift(1) * temp['log_return'].values.reshape(-1, 1)
    temp.dropna(inplace = True) 

    return np.exp(temp[['log_return', 'returne']].sum())


In [4]:
'''
BENCHMARK
'''

def benchmark_accuracy(prediction, test):
    prediction = pd.DataFrame(prediction)
    test = pd.DataFrame(test)

    return (prediction == test).sum() / len(prediction)


In [5]:
''' 
MODEL
'''

def predict_position(train, test):
    # Train Hidden Markov Model
    model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=1000)
    model.fit(train.reshape(-1, 1))

    # Predict hidden states for test data
    hidden_states = model.predict(test.reshape(-1, 1))

    # Define a function to map hidden states to rise or fall
    def map_to_rise_or_fall(state):
        return 1 if state == 0 else -1

    # Map hidden states to rise or fall
    predicted_rise_fall = np.array(list(map(map_to_rise_or_fall, hidden_states)))

    return predicted_rise_fall

In [6]:
apple_data = pd.read_csv('gs_output.csv', index_col = 0, parse_dates = True).dropna()
apple_data['SMA_EMA_ratio'] = apple_data['SMA'] / apple_data['EMA']
SMA_short = 7
SMA_long = 20
apple_data['SMA_Short'] = apple_data['GS.N'].rolling(SMA_short).mean()
apple_data['SMA_Long'] = apple_data['GS.N'].rolling(SMA_long).mean()
apple_data['SMA_Long_Short'] = apple_data['SMA_Short'] / apple_data['SMA_Long']
apple_data['data'] = apple_data['GS.N']
apple_data['direction'] = np.where(apple_data['GS.N'] <= apple_data['GS.N'].shift(-1), 1, -1)

apple_data

,GS.N,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data,direction
Date,,,,,,,,,,,,,,,,,,
2010-04-01,173.08,9.7641,15.2302,50.3525,25.5682,-119.6258,100.0000,64.2857,-177810600.0,8.118063e+08,137.0128,135.9827,1.007575,NaN,NaN,NaN,173.08,1
2010-06-01,174.26,82.1737,80.0595,49.2538,31.4835,85.6424,71.4286,14.2857,-346418300.0,6.644775e+08,110.7708,112.1395,0.987795,NaN,NaN,NaN,174.26,1
2010-07-01,177.67,9.8352,26.4734,37.8543,33.0676,-128.0550,100.0000,71.4286,-430663700.0,6.215069e+08,107.1164,106.6629,1.004252,NaN,NaN,NaN,177.67,-1
2010-11-01,171.56,75.5784,75.1963,67.9799,37.1642,90.3937,14.2857,85.7143,-336975200.0,6.427520e+08,125.3926,125.2671,1.001002,NaN,NaN,NaN,171.56,-1
2010-12-01,167.82,28.6289,20.2051,39.2460,29.1709,-57.8186,92.8571,21.4286,-340198800.0,6.448263e+08,127.3640,126.7452,1.004882,NaN,NaN,NaN,167.82,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,221.54,19.9620,20.9447,32.7234,22.7421,-152.1742,100.0000,42.8571,-240254198.0,6.437021e+08,201.4811,200.4351,1.005219,227.712857,230.5165,0.987838,221.54,1
2018-06-26,221.58,17.8048,19.8747,28.5329,24.4674,-140.4348,100.0000,35.7143,-237561311.0,6.446095e+08,200.4011,199.2830,1.005611,226.235714,229.7905,0.984530,221.58,-1
2018-06-27,220.18,14.1021,17.2896,28.0921,24.3184,-105.2860,92.8571,28.5714,-240664932.0,6.415525e+08,199.3360,198.2927,1.005261,224.634286,229.0490,0.980726,220.18,1


In [7]:
scaler = MinMaxScaler()

# Normalize the selected columns
normalized_columns = scaler.fit_transform(apple_data.iloc[:, :-2])

# Create a new DataFrame with the normalized values
normalized_df = pd.DataFrame(normalized_columns, columns=apple_data.columns[:-2])

# Display the normalized DataFrame
apple_data.iloc[:, :-2] = normalized_df
apple_data


,GS.N,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data,direction
Date,,,,,,,,,,,,,,,,,,
2010-04-01,0.459823,0.082321,0.137138,0.475208,0.226894,0.311874,1.000000,0.642857,0.895429,0.940148,0.398379,0.389250,0.662791,NaN,NaN,NaN,173.08,1
2010-06-01,0.466178,0.833234,0.832238,0.462212,0.311360,0.676722,0.714286,0.142857,0.453773,0.484560,0.234722,0.239687,0.270149,NaN,NaN,NaN,174.26,1
2010-07-01,0.484543,0.083058,0.257687,0.327378,0.333980,0.296891,1.000000,0.714286,0.233098,0.351682,0.211932,0.205333,0.596819,NaN,NaN,NaN,177.67,-1
2010-11-01,0.451637,0.764839,0.780095,0.683706,0.392476,0.685167,0.142857,0.857143,0.478508,0.417378,0.325910,0.322033,0.532310,NaN,NaN,NaN,171.56,-1
2010-12-01,0.431495,0.277956,0.190479,0.343839,0.278338,0.421731,0.928571,0.214286,0.470065,0.423793,0.338205,0.331305,0.609335,NaN,NaN,NaN,167.82,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.720810,0.188077,0.198409,0.266690,0.186540,0.254021,1.000000,0.428571,0.731862,0.420316,0.800433,0.793548,0.616013,0.768707,0.796155,0.422114,221.54,1
2018-06-26,0.721025,0.165706,0.186936,0.217124,0.211175,0.274887,1.000000,0.357143,0.738916,0.423122,0.793698,0.786321,0.623793,0.760472,0.791935,0.405625,221.58,-1
2018-06-27,0.713486,0.127308,0.159219,0.211910,0.209048,0.337361,0.928571,0.285714,0.730786,0.413669,0.787055,0.780109,0.616862,0.751543,0.787625,0.386658,220.18,1


In [8]:
# apple_data = apple_data[['SlowK', 'RSI', 'CCI', 'Aroon Down', 'OBV', 'direction']]

In [9]:
train_data, test_data = sequential_train_test_split(apple_data)
print(test_data)
X_train, y_train, X_test, y_test = train_data[:, :-1], train_data[:, -1], test_data[:, :-1], test_data[:, -1]

[[ 4.70809996e-01  8.55688192e-01  8.90355660e-01 ...  6.39738955e-01
   1.75120000e+02  1.00000000e+00]
 [ 4.73125808e-01  8.74889945e-01  8.88052572e-01 ...  6.49533555e-01
   1.75550000e+02  1.00000000e+00]
 [ 4.81311935e-01  8.74405649e-01  8.91189832e-01 ...  6.65372309e-01
   1.77070000e+02  1.00000000e+00]
 ...
 [ 7.13485567e-01  1.27307534e-01  1.59218623e-01 ...  3.86657519e-01
   2.20180000e+02  1.00000000e+00]
 [ 7.30934942e-01  2.18604005e-01  1.69740131e-01 ...  3.75264786e-01
   2.23420000e+02 -1.00000000e+00]
 [ 7.15585954e-01  2.22813333e-01  1.89421451e-01 ...  3.61831907e-01
   2.20570000e+02 -1.00000000e+00]]


In [10]:
X_train

array([[  0.34144765,   0.84059932,   0.75434697, ...,   0.3983534 ,
          0.19772892, 151.1       ],
       [  0.34893365,   0.42260033,   0.2256404 , ...,   0.39236975,
          0.23489945, 152.49      ],
       [  0.3573352 ,   0.74073901,   0.78156372, ...,   0.38649652,
          0.27616486, 154.05      ],
       ...,
       [  0.46752477,   0.75832714,   0.6569536 , ...,   0.42398519,
          0.60584503, 174.51      ],
       [  0.46752477,   0.88157053,   0.7947282 , ...,   0.42639725,
          0.60795012, 174.51      ],
       [  0.46838647,   0.86530255,   0.85680113, ...,   0.42878025,
          0.61939288, 174.67      ]])

In [11]:
from sklearn.svm import SVC
model = SVC(kernel='rbf', gamma=0.7, C=1.0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
benchmark_accuracy(pred, y_test)

0    0.467914
dtype: float64

In [12]:
best_acc = 0
best_g = -1
best_c = -1
best_return = 0
bestgg = -1
bestcc = -1
log_return = 0  

selected_features = [4, 5, 9]
dat = X_test[:,-1]

for g in range(1, 11, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='rbf', gamma=g / 10, C=c / 10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp = calculate_return(pred, dat)
        log_return = temp.values[0]
        if acc[0] > best_acc:
            best_acc = acc[0]
            best_g = g
            best_c = c
        if temp.values[1] > best_return :
            best_return = temp.values[1]
            bestgg = g
            bestcc = c

print(f"best acc: {best_acc}, g: {best_g}, c: {best_c}")
print(f"best ret: {best_return}, g: {bestgg}, c: {bestcc}")

best acc: 0.516042780748663, g: 10, c: 43
best ret: 1.442167130366157, g: 10, c: 43


In [13]:
print(log_return)

1.2595363179534016


In [14]:
best_acc = 0
best_c = -1
best_return = 0
bestcc = -1

selected_features = [4, 5, 9]
dat = X_test[:,-1]

for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='sigmoid', C= c/10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp = calculate_return(pred, dat)

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_c = c
    if temp.values[1] > best_return :
        best_return = temp.values[1]
        bestcc = c
    
print(f"best acc: {best_acc}, c: {best_c}")
print(f"best ret: {best_return}, c: {bestcc}")

best acc: 0.5026737967914439, c: 1
best ret: 1.2595363179534016, c: 1


In [15]:
best_acc = 0
best_d = -1
best_c = -1
best_return = 0
bestdd = -1
bestcc = -1

selected_features = [1, 5, 11, 13]
dat = X_test[:,-1]

for d in range(1, 5, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree=d, C=c/10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp = calculate_return(pred, dat)

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_d = d
            best_c = c
        if temp.values[1] > best_return :
            best_return = temp.values[1]
            bestdd = d
            bestcc = c
    
print(f"best acc: {best_acc}, d: {best_d}, c: {best_c}")
print(f"best ret: {best_return}, d : {bestdd}, c: {bestcc}")

best acc: 0.5374331550802139, d: 4, c: 27
best ret: 1.773358243285175, d : 4, c: 4


In [16]:
best_acc = 0
best_c = -1
best_return = 0
bestcc = -1

selected_features = [4, 5, 9]

# for d in range(1, 5, 1):
for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='poly', degree=2, C = c / 10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp = calculate_return(pred, dat)

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_c = c
    if temp.values[1] > best_return :
        best_return = temp.values[1]
        bestcc = c
    
print(f"best acc: {best_acc}, c: {best_c}")
print(f"best ret: {best_return}, c: {bestcc}")

best acc: 0.5053475935828877, c: 3
best ret: 1.2813480977766836, c: 3


In [17]:
# Feature Engineering

# Feature Selection

In [18]:
X = apple_data.dropna().iloc[:, :-1]
y = apple_data.dropna().iloc[:, -1]

In [19]:
X

,GS.N,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data
Date,,,,,,,,,,,,,,,,,
2010-08-02,0.341448,0.840599,0.754347,0.655482,0.309261,0.775527,0.071429,1.000000,0.381960,0.386987,0.272273,0.268584,0.520194,0.351097,0.398353,0.197729,151.10
2010-09-02,0.348934,0.422600,0.225640,0.340999,0.471636,0.437559,0.857143,0.000000,0.357879,0.351465,0.243308,0.237787,0.567587,0.352379,0.392370,0.234899,152.49
2010-11-02,0.357335,0.740739,0.781564,0.690783,0.403324,0.701184,0.071429,0.785714,0.487237,0.424592,0.329316,0.325033,0.542969,0.354410,0.386497,0.276165,154.05
2010-12-02,0.356689,0.480375,0.291657,0.383895,0.243258,0.567030,0.857143,0.142857,0.488947,0.438502,0.335447,0.330330,0.564498,0.354920,0.379597,0.314747,153.93
2010-02-16,0.375377,0.848106,0.749272,0.446052,0.362687,0.699900,0.214286,0.428571,0.802486,0.826197,0.296154,0.294417,0.472482,0.361833,0.375482,0.375411,157.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.720810,0.188077,0.198409,0.266690,0.186540,0.254021,1.000000,0.428571,0.731862,0.420316,0.800433,0.793548,0.616013,0.768707,0.796155,0.422114,221.54
2018-06-26,0.721025,0.165706,0.186936,0.217124,0.211175,0.274887,1.000000,0.357143,0.738916,0.423122,0.793698,0.786321,0.623793,0.760472,0.791935,0.405625,221.58
2018-06-27,0.713486,0.127308,0.159219,0.211910,0.209048,0.337361,0.928571,0.285714,0.730786,0.413669,0.787055,0.780109,0.616862,0.751543,0.787625,0.386658,220.18


In [20]:
y

Date
2010-08-02    1
2010-09-02    1
2010-11-02   -1
2010-12-02    1
2010-02-16   -1
             ..
2018-06-25    1
2018-06-26   -1
2018-06-27    1
2018-06-28   -1
2018-06-29   -1
Name: direction, Length: 1868, dtype: int32

In [21]:
best_acc = 0
best_ret = 0

dat = X_test[:,-1]
for i in range(len(apple_data.columns) - 1):
    print(X_train[:,i])
    curr_X_train = X_train[:,i].reshape(-1, 1)
    curr_X_test = X_test[:,i].reshape(-1, 1)
    model = SVC(kernel='rbf', gamma= 1.0, C= 1.0)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp =calculate_return(pred, dat).values[1]
    if temp > best_ret:
        best_ret = temp
    
    print(f"feature {i}: acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")
print(best_ret)

[0.34144765 0.34893365 0.3573352  ... 0.46752477 0.46752477 0.46838647]
feature 0: acc: 0.503 up: 1.0
[0.84059932 0.42260033 0.74073901 ... 0.75832714 0.88157053 0.86530255]
feature 1: acc: 0.503 up: 1.0
[0.75434697 0.2256404  0.78156372 ... 0.6569536  0.7947282  0.85680113]
feature 2: acc: 0.503 up: 1.0
[0.65548166 0.34099949 0.69078251 ... 0.67221718 0.68790119 0.62648871]
feature 3: acc: 0.503 up: 1.0
[0.30926094 0.47163574 0.40332392 ... 0.21737982 0.23246297 0.23092795]
feature 4: acc: 0.503 up: 1.0
[0.77552663 0.43755897 0.7011838  ... 0.81975175 0.76412411 0.67988797]
feature 5: acc: 0.503 up: 1.0
[0.071429 0.857143 0.071429 ... 0.       0.       0.      ]
feature 6: acc: 0.503 up: 1.0
[1.       0.       0.785714 ... 1.       1.       0.928571]
feature 7: acc: 0.503 up: 1.0
[0.38196025 0.35787852 0.48723748 ... 0.67227221 0.67227221 0.67752246]
feature 8: acc: 0.503 up: 1.0
[0.38698681 0.35146547 0.42459183 ... 0.46929933 0.46711055 0.4721157 ]
feature 9: acc: 0.503 up: 1.0
[0.2

In [22]:
best_acc = 0
best_selected_features = []
best_ret = 0
best_selected_featuresa = []

for i in range(len(apple_data.columns) - 1):
    for j in range(len(apple_data.columns) - 1):
        selected_features = [i, j]
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp =calculate_return(pred, dat).values[1]
        if temp > best_ret:
            best_ret = temp
            best_selected_featuresa = selected_features

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")

best acc: 0.5320855614973262 features: [7, 11]
best ret: 1.782160715098622 features: [7, 11]


In [23]:
print(apple_data.columns)

Index(['GS.N', 'SlowK', 'SlowD', 'RSI', 'ADX', 'CCI', 'Aroon Down', 'Aroon Up',
       'OBV', 'Chaikin A/D', 'SMA', 'EMA', 'SMA_EMA_ratio', 'SMA_Short',
       'SMA_Long', 'SMA_Long_Short', 'data', 'direction'],
      dtype='object')


In [32]:
import itertools

all_features = list(range(len(apple_data.columns) - 2))

best_acc = 0
best_selected_features = []
best_ret = 0
best_selected_featuresa = []

for i in range(1, len(apple_data.columns) - 2):
    print(i)
    possible_selected_features = list(itertools.combinations(all_features, i))
    for selected_features in possible_selected_features:
        print(selected_features)
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp =calculate_return(pred, dat).values[1]
        if temp > best_ret:
            best_ret = temp
            best_selected_featuresa = selected_features

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")

1
(0,)
(1,)
(2,)
(3,)
(4,)
(5,)
(6,)
(7,)
(8,)
(9,)
(10,)
(11,)
(12,)
(13,)
(14,)
(15,)
2
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(1, 13)
(1, 14)
(1, 15)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(2, 8)
(2, 9)
(2, 10)
(2, 11)
(2, 12)
(2, 13)
(2, 14)
(2, 15)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(3, 8)
(3, 9)
(3, 10)
(3, 11)
(3, 12)
(3, 13)
(3, 14)
(3, 15)
(4, 5)
(4, 6)
(4, 7)
(4, 8)
(4, 9)
(4, 10)
(4, 11)
(4, 12)
(4, 13)
(4, 14)
(4, 15)
(5, 6)
(5, 7)
(5, 8)
(5, 9)
(5, 10)
(5, 11)
(5, 12)
(5, 13)
(5, 14)
(5, 15)
(6, 7)
(6, 8)
(6, 9)
(6, 10)
(6, 11)
(6, 12)
(6, 13)
(6, 14)
(6, 15)
(7, 8)
(7, 9)
(7, 10)
(7, 11)
(7, 12)
(7, 13)
(7, 14)
(7, 15)
(8, 9)
(8, 10)
(8, 11)
(8, 12)
(8, 13)
(8, 14)
(8, 15)
(9, 10)
(9, 11)
(9, 12)
(9, 13)
(9, 14)
(9, 15)
(10, 11)
(10, 12)
(10, 13)
(10, 14)
(10, 15)
(11, 12)
(11, 13)
(11, 14)
(11, 15)
(12, 13)
(12, 14)
(12, 15)
(13, 14

KeyboardInterrupt: 

In [34]:

print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")

best acc: 0.5668449197860963 features: (0, 10)
best ret: 2.3992565838150837 features: (0, 2, 6, 8, 9, 10, 12)
